<a href="https://colab.research.google.com/github/angelinawong1210/AiCOVID/blob/main/LinearProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# Linear programming and Optimization using **Scipy** in Python 



---
The structure of file **input.txt** must follow these rules:

- The first line is the **Objective function** and **NA** in the end of the line is to fill up the blanks

- The following lines are **Constraint Function**: $Ax = b$ or $Ax \leq b$ or $Ax \geq b$

- File **input.txt** may look like these: 

In [ ]:
with open("input.txt") as f:
    for line in f.readlines():
        print(line, end="")
    f.close()

10 15 25 min NA
-1 -1 -1 >= -1000
-1 2 0 >= 0
0 0 -1 <= -340
-1 0 0 = 0
-1 0 0 >= 0
0 0 -1 <= 0

---
First, read the file **input.txt** and convert it into the table using DataFrame as followed: (let the variables be **df**)

In [ ]:
df = pd.read_table("input.txt", header=None, sep="\s+")
df

,0,1,2,3,4
0,10,15,25,min,NaN
1,-1,-1,-1,>=,-1000.0
2,-1,2,0,>=,0.0
3,0,0,-1,<=,-340.0
4,-1,0,0,=,0.0
5,-1,0,0,>=,0.0
6,0,0,-1,<=,0.0


---
To make it is possible for the function to be executed, we have the following conventional:

_max_ <center>$cx$</center>
when <center> $Ax \leq b$ </center>

or:

_min_ <center>$cx$</center>
when <center> $Ax \geq b$ </center>

The **negative** function will convert the **Constraint** from negative to positive or in reversed. 

In [ ]:
def negative(x):
    if type(x) == str:
        if x == "<=":
            x = ">="
        else:
            x = "<="
    else:
        x = x * (-1)
    return x

- Next, we have the **inputTransfrom** function to make sure that the input data followed our conventional

In [ ]:
def inputTransfrom(df):

    # nếu data chưa đúng format thì sẽ được xử lý lại
    col = len(df.columns)-2

    for i in range(1, len(df.index)):
        if (df[col][i] == ">=" and df[3][0] == "max"):
            df.loc[i] = df.loc[i].apply(negative).values
        elif (df[col][i] == "<=" and df[3][0] == "min"):
            df.loc[i] = df.loc[i].apply(negative).values


    return df

In [ ]:
df = inputTransfrom(df)
df

,0,1,2,3,4
0,10,15,25,min,NaN
1,-1,-1,-1,>=,-1000.0
2,-1,2,0,>=,0.0
3,0,0,1,>=,340.0
4,-1,0,0,=,0.0
5,-1,0,0,>=,0.0
6,0,0,1,>=,-0.0


Next, append the needed value of $c$, $A$, $b$, using ```.iloc```

For the values of $A$, we divide into two cases:

*   If  $Ax = b$, assign $A_{eq}$ 
*   In the other cases, assign  $A_{ub}$



In [ ]:
col = len(df.columns) - 2

c = df.iloc[0, :3].values
b = df.iloc[1:, 4].values

A_ub = df[~(df[col] == "=")].iloc[1:-1, 0:col].values
b_ub = df[~(df[col] == "=")].iloc[1:-1, 4].values

where_equal = list(df[df[col]=="="].index)
A_eq = df[df[col] == "="].loc[where_equal].iloc[:, 0:col].values
b_eq = df[df[col] == "="].iloc[:, -1].values

In [ ]:
print(c)
print(A_eq)
print(b_eq)
print(A_ub)
print(b_ub)

[10 15 25]
[[-1  0  0]]
[0.]
[[-1 -1 -1]
 [-1  2  0]
 [ 0  0  1]
 [-1  0  0]]
[-1000.     0.   340.     0.]


- The final problem is to find the conditions for $x$ and declare the variables ```_bounds``` as followed (e.g if we have $x_1, x_2, x_3 \geq 0 $, **None** represent $\infty$ or -$\infty$ )

In [ ]:
x1_bounds = (0, None)
x2_bounds = (0, None)
x3_bounds = (0, None)

- Finally, we have variables needed to apply the function ```linprog``` of **Scipy** to solve the linear programming and optimization

In [ ]:
res = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds = [x1_bounds, x2_bounds, x3_bounds])

In [ ]:
print(res)

     con: array([0.])
     fun: 21.163655289804403
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 3
   slack: array([-998.73526158,   -2.09096105,  339.7807421 ,    0.        ])
  status: 2
 success: False
       x: array([-0.        ,  1.04548052,  0.2192579 ])


- **message**: The algorithm terminated successfully and determined that the problem is infeasible
-  **success: False** means there is no solution for this problem 
- **x** is the approximate solutions 